## Carga modulo comun

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../common'))

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from purrfect.dataset import load_partition,save_partition, create_train_valid_loaders

from purrfect.training import train_model
import torch.optim as optim
from purrfect.active_learning import create_next_partitions, test_model

from sklearn.model_selection import train_test_split
from purrfect.submission import create_submission

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Definición modelo

In [4]:
class ChannelAdder(nn.Module):
    def __init__(self):
        super(ChannelAdder, self).__init__()
        # Define Sobel and Laplacian kernels as tensors
        self.sobel_x = torch.tensor([[-1., 0., 1.],
                                     [-2., 0., 2.],
                                     [-1., 0., 1.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)
        
        self.sobel_y = torch.tensor([[-1., -2., -1.],
                                     [ 0.,  0.,  0.],
                                     [ 1.,  2.,  1.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)
        
        self.laplacian_kernel = torch.tensor([[0.,  1., 0.],
                                              [1., -4., 1.],
                                              [0.,  1., 0.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)

    def forward(self, x):
        epsilon = 1e-8

        # Extract the first, second, and third channels
        first_channel = x[:, 0, :, :].unsqueeze(1)  # e1 (first channel)
        second_channel = x[:, 1, :, :].unsqueeze(1)  # e2 (second channel)
        third_channel = x[:, 2, :, :].unsqueeze(1)  # error (third channel)

        # 1. Compute the first new channel: sqrt(first_channel^2 + second_channel^2)
        new_channel1 = torch.sqrt(first_channel**2 + second_channel**2)

        # 2. Compute the second new channel: 1/2 * arctan(channel2 / channel1)
        new_channel2 = 0.5 * torch.atan(second_channel / (first_channel + epsilon))

        # 3. Compute Sobel gradients and Laplacians for e1 (first_channel)
        grad_e1_x = F.conv2d(first_channel, self.sobel_x, padding=1)
        grad_e1_y = F.conv2d(first_channel, self.sobel_y, padding=1)
        grad_e1_magnitude = torch.sqrt(grad_e1_x**2 + grad_e1_y**2)
        #laplacian_e1 = F.conv2d(first_channel, self.laplacian_kernel, padding=1)

        # 4. Compute Sobel gradients and Laplacians for e2 (second_channel)
        grad_e2_x = F.conv2d(second_channel, self.sobel_x, padding=1)
        grad_e2_y = F.conv2d(second_channel, self.sobel_y, padding=1)
        grad_e2_magnitude = torch.sqrt(grad_e2_x**2 + grad_e2_y**2)
        #laplacian_e2 = F.conv2d(second_channel, self.laplacian_kernel, padding=1)

        # 5. Compute weighted ellipticity channels (e1_weighted, e2_weighted)
        e1_weighted = first_channel / (third_channel + epsilon)
        e2_weighted = second_channel / (third_channel + epsilon)

        # Concatenate all the channels (original and new) into the output tensor
        output = torch.cat([
            x,                 # Original 3 channels
            new_channel1,      # sqrt(channel1^2 + channel2^2)
            new_channel2,      # 1/2 * arctan(channel2 / channel1)
            #grad_e1_x,         # Gradient X of channel1
            #grad_e1_y,         # Gradient Y of channel1
            #grad_e2_x,         # Gradient X of channel2
            #grad_e2_y,         # Gradient Y of channel2
            grad_e1_magnitude, # Gradient magnitude of channel1
            grad_e2_magnitude, # Gradient magnitude of channel2
            #laplacian_e1,      # Laplacian of channel1
            #laplacian_e2,      # Laplacian of channel2
            e1_weighted,       # e1_weighted
            e2_weighted        # e2_weighted
        ], dim=1)

        return output

# Define the Inception block with Dilated Convolutions
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionBlock, self).__init__()
        # 1x1 convolution
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(out_channels)
        )

        # 3x3 convolution
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=2, dilation=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(out_channels)
        )

        # 5x5 convolution
        self.branch5 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=4, dilation=4),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(out_channels)
        )

        self.cast = nn.Conv2d(out_channels*3, out_channels, kernel_size=1)

    def forward(self, x):
        # Apply each branch
        branch1 = self.branch1(x)
        branch3 = self.branch3(x)
        branch5 = self.branch5(x)

        # Concatenate the outputs along the channel dimension
        outputs = torch.cat([branch1, branch3, branch5], dim=1)
        return self.cast(outputs)


# Define Additive Attention
class AttentionBlock(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(AttentionBlock, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(F_int),
        )
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(F_int)
        )
        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = F.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi

# U-Net with Additive Attention, Inception, and Recurrence
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.channel_adder = ChannelAdder()
        self.encoder1 = nn.Sequential(
            InceptionBlock(in_channels+6, 64),
            nn.LeakyReLU(inplace=True),
        )
        self.encoder2 = nn.Sequential(
            nn.MaxPool2d(2),
            InceptionBlock(64, 128),
            nn.LeakyReLU(inplace=True),
        )
        self.encoder3 = nn.Sequential(
            nn.MaxPool2d(2),
            InceptionBlock(128, 256),
            nn.LeakyReLU(inplace=True),
        )
        self.encoder4 = nn.Sequential(
            nn.MaxPool2d(2),
            InceptionBlock(256, 512),
            nn.LeakyReLU(inplace=True),
        )
        self.center = nn.Sequential(
            nn.MaxPool2d(2),
            InceptionBlock(512, 1024),
            nn.LeakyReLU(inplace=True),
        )

        self.decoder4 = nn.Sequential(
            nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(inplace=True),
            InceptionBlock(512, 512),
            nn.LeakyReLU(inplace=True),
        )
        self.att4 = AttentionBlock(F_g=512, F_l=512, F_int=256)

        self.decoder3 = nn.Sequential(
            nn.ConvTranspose2d(2*512, 256, kernel_size=2, stride=2),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(inplace=True),
            InceptionBlock(256, 256),
            nn.LeakyReLU(inplace=True),
        )
        self.att3 = AttentionBlock(F_g=256, F_l=256, F_int=128)

        self.decoder2 = nn.Sequential(
            nn.ConvTranspose2d(2*256, 128, kernel_size=2, stride=2),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(inplace=True),
            InceptionBlock(128, 128),
            nn.LeakyReLU(inplace=True),
        )
        self.att2 = AttentionBlock(F_g=128, F_l=128, F_int=64)

        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(2*128, 64, kernel_size=2, stride=2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            InceptionBlock(64, 64),
            nn.LeakyReLU(inplace=True),
        )
        self.final = nn.Conv2d(2*64, out_channels, kernel_size=1)

    def forward(self, x):
        x = self.channel_adder(x)
        # Encoder path
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)
        center = self.center(e4)

        # Decoder path
        d4 = self.decoder4(center)
        e4 = self.att4(g=d4, x=e4)
        d4 = torch.cat([e4, d4], dim=1)

        d3 = self.decoder3(d4)
        e3 = self.att3(g=d3, x=e3)
        d3 = torch.cat([e3, d3], dim=1)

        d2 = self.decoder2(d3)
        e2 = self.att2(g=d2, x=e2)
        d2 = torch.cat([e2, d2], dim=1)

        d1 = self.decoder1(d2)
        d1 = torch.cat([e1, d1], dim=1)

        out = self.final(d1)

        return out

## Creación particion inicial

In [5]:
#Creacion de particiones train y valid
init_partition = load_partition("partition_1.json")
train_partition, val_partition = train_test_split(init_partition, test_size=0.2, random_state=42)
save_partition("partition_1_train.json","partitions",train_partition)
save_partition("partition_1_val.json","partitions",val_partition)

## Carga modelo

In [6]:
#Define model
model = UNet( 3, 1)
model = model.to(DEVICE)

In [7]:
# Define Loss
criterion = torch.nn.L1Loss()
current_partition = 1

In [8]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:12<00:00,  6.19it/s, WMAPE=4.23, DICE=0.166, DPEAKS=111, Loss=0.0132]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:12<00:00,  6.16it/s, WMAPE=4.99, DICE=0.11, DPEAKS=62, Loss=0.0148]


Epoch [3/50]


...

Epoch [6/50]


Validate Epoch 6: 100%|██████████| 77/77 [00:12<00:00,  6.15it/s, WMAPE=4.8, DICE=0.0779, DPEAKS=44.1, Loss=0.0349]


Epoch [7/50]


Validate Epoch 7: 100%|██████████| 77/77 [00:12<00:00,  6.15it/s, WMAPE=7.22, DICE=0.0731, DPEAKS=37.4, Loss=0.0176]


early stopping: 3 epochs without improvement
Training complete.


In [9]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)

Validate Epoch test: 100%|██████████| 1941/1941 [05:08<00:00,  6.29it/s, WMAPE=5.03, DICE=0.0914, DPEAKS=58.8, Loss=0.0238]


In [10]:
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_2_val: 100%|██████████| 77/77 [00:12<00:00,  6.30it/s, WMAPE=2.57, DICE=0.0914, DPEAKS=61.7, Loss=0.00786]


In [11]:
current_partition=2

In [12]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Train Epoch 1:   0%|          | 0/307 [00:00<?, ?it/s]

Validate Epoch 1: 100%|██████████| 77/77 [00:11<00:00,  6.62it/s, WMAPE=2.76, DICE=0.0974, DPEAKS=74.2, Loss=0.00802]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:12<00:00,  6.15it/s, WMAPE=2.37, DICE=0.0729, DPEAKS=51, Loss=0.00762]


...

Epoch [4/50]


Validate Epoch 4: 100%|██████████| 77/77 [00:11<00:00,  6.66it/s, WMAPE=9.73, DICE=0.0808, DPEAKS=47.1, Loss=0.0659]


Epoch [5/50]


Validate Epoch 5: 100%|██████████| 77/77 [00:12<00:00,  6.11it/s, WMAPE=485, DICE=0.0734, DPEAKS=47.6, Loss=2.45]


early stopping: 3 epochs without improvement
Training complete.


In [13]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_3_val: 100%|██████████| 77/77 [00:12<00:00,  6.34it/s, WMAPE=1.96, DICE=0.0714, DPEAKS=43.1, Loss=0.00634]


In [14]:
for current_partition in range(3,5):
    best_model_path = os.path.join(
        "models", f"best_model_partition_{current_partition}.pth"
    )
    last_checkpoint_path = os.path.join(
        "models", f"last_checkpoint_partition_{current_partition}.pth"
    )
    train_loader, val_loader = create_train_valid_loaders(
        f"partition_{current_partition}_train.json",
        f"partition_{current_partition}_val.json",
        "partitions",
        batch_size=16,
    )
    optimizer = optim.Adam(model.parameters())
    train_model(
        model,
        train_loader,
        val_loader,
        best_model_path,
        last_checkpoint_path,
        criterion,
        optimizer,
        num_epochs=50,
        device=DEVICE,
        early_stopping_patience=3,
    )
    #Cargar mejor modelo de la particion actual
    model.load_state_dict(torch.load(best_model_path,weights_only=True))
    test_model(model,criterion,device=DEVICE,batch_size=16)
    create_next_partitions(current_partition,model,criterion,device=DEVICE)

Epoch [1/50]


Train Epoch 1:   0%|          | 0/307 [00:00<?, ?it/s]

Validate Epoch 1: 100%|██████████| 77/77 [00:12<00:00,  6.20it/s, WMAPE=3.87, DICE=0.143, DPEAKS=100, Loss=0.0124]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:12<00:00,  6.17it/s, WMAPE=362, DICE=0.0801, DPEAKS=55, Loss=0.122]


...

Validate Epoch 3: 100%|██████████| 77/77 [00:12<00:00,  6.25it/s, WMAPE=9.52e+3, DICE=0.0723, DPEAKS=57, Loss=0.268]


Epoch [4/50]


Validate Epoch 4: 100%|██████████| 77/77 [00:12<00:00,  6.14it/s, WMAPE=1.34e+4, DICE=0.15, DPEAKS=92.6, Loss=76.1]


early stopping: 3 epochs without improvement
Training complete.


Validate Epoch partition_5_val: 100%|██████████| 77/77 [00:12<00:00,  6.15it/s, WMAPE=4.94, DICE=0.0805, DPEAKS=40.2, Loss=0.0139]
